In [ ]:
%pip install datasets
%pip install librosa

In [1]:
from datasets import load_dataset
ds = load_dataset("diarizers-community/ami", "ihm")

print(ds)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['audio', 'timestamps_start', 'timestamps_end', 'speakers'],
        num_rows: 136
    })
    validation: Dataset({
        features: ['audio', 'timestamps_start', 'timestamps_end', 'speakers'],
        num_rows: 18
    })
    test: Dataset({
        features: ['audio', 'timestamps_start', 'timestamps_end', 'speakers'],
        num_rows: 16
    })
})


In [ ]:
ds['train'][0]

## Get wav file audio

In [41]:
import soundfile as sf

def export_as_wav_file(ds_entry, output_path = 'test.wav'):
    y = ds_entry['audio']['array']
    sr = ds_entry['audio']['sampling_rate']

    sf.write(output_path, y, sr)

ds_entry = ds['train'][0]
export_as_wav_file(ds_entry)

In [34]:
def find_unique_elements(lst):
    return list(set(lst))

unique_elements = find_unique_elements(ds['train'][0]['speakers'])

print(f"Number of speakers = {len(unique_elements)}")
unique_elements

Number of speakers = 4


['MIO026', 'MIE029', 'MIE083', 'MIE080']

## Get rttm file as ground truth

In [40]:
def export_rttm_file(df, file_id, output_path = 'test.rttm'):
    ''' Get rttm of 1 entry in ami'''

    timestamps_start = df['timestamps_start']
    timestamps_end = df['timestamps_end']
    speakers = df['speakers']

    for i in range(len(timestamps_start)):

        duration = timestamps_end[i] - timestamps_start[i]

        with open(output_path, 'a') as f:
            line = f"SPEAKER {file_id} 1 {timestamps_start[i]:.3f} {duration:.3f} <NA> <NA> {speakers[i]} <NA>\n"
            f.write(line)


export_rttm_file(ds['train'][0], ds['train'][0]['audio']['path'], 'test.rttm')

## Exporting out the entire dataset

In [3]:
ds['train'][0]['audio']['path']

'IS1002c.Mix-Headset.wav'

In [9]:
import soundfile as sf
import os

def export_as_wav_file(ds_entry, output_path = 'test.wav'):
    ''' export 1 entry in ami'''
    y = ds_entry['audio']['array']
    sr = ds_entry['audio']['sampling_rate']

    sf.write(output_path, y, sr)

def export_rttm_file(df, file_id, output_path = 'test.rttm'):
    ''' Get rttm of 1 entry in ami'''

    timestamps_start = df['timestamps_start']
    timestamps_end = df['timestamps_end']
    speakers = df['speakers']

    for i in range(len(timestamps_start)):

        duration = timestamps_end[i] - timestamps_start[i]

        with open(output_path, 'a') as f:
            line = f"SPEAKER {file_id} 1 {timestamps_start[i]:.3f} {duration:.3f} <NA> <NA> {speakers[i]} <NA>\n"
            f.write(line)

count = 0
for entry in ds['validation']:

    basename, _ = os.path.splitext(entry['audio']['path'])

    wav_output_path = 'ami_dataset/audio/' + basename +'.wav'
    rttm_output_path = 'ami_dataset/rttm/' + basename +'.rttm'

    export_as_wav_file(entry, wav_output_path)
    export_rttm_file(entry, basename, rttm_output_path)

    count+=1
    if count % 10 == 0:
        print(f"We are at {count}!")


We are at 10!
